# Análisis estadístico básico de los grupos
En este notebook, se verá un análisis general para los grupos creados, esto con la intención de entender a grandes rasgos la composición de cada grupo

In [1]:
import sys
sys.path.insert(0, './scripts/')

In [2]:
from collections import defaultdict
import gc
import IDTMex
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st


In [3]:
gc.collect()

0

In [4]:
pd.set_option('display.max_columns', None)

### Lectura de Datos

In [5]:
base_path = "./ConjuntosDatos/"
endutih_path = base_path + "ENDUTIH2020/conjuntos_de_datos/"

In [6]:
endutih_vivhogar = pd.read_csv(endutih_path+"viv_hog_agrupados.csv")
endutih_res = pd.read_csv(endutih_path+"residente_agrupados.csv")
endutih_usr = pd.read_csv(endutih_path+"usuario_agrupados.csv", low_memory=False)
endutih_usr2 = pd.read_csv(endutih_path+"usuario2_agrupados.csv", low_memory=False)
endutih_completa = pd.merge(endutih_vivhogar,endutih_res.drop(["UPM_DIS","ESTRATO","ENT","DOMINIO", "EST_DIS", "TLOC","Grupo"], axis=1), on = ["UPM","VIV_SEL", "HOGAR"])

In [7]:
estados = pd.read_csv(base_path+"CodigoEdos.csv")
estados

,Codigo,Nombre,ENT
0,1,Aguascalientes,Ags.
1,2,Baja California,B.C.
2,3,Baja California Sur,B.C.S
3,4,Campeche,Camp.
4,5,Coahuila de Zaragoza,Coah.
5,6,Colima,Col.
6,7,Chiapas,Chis.
7,8,Chihuahua,Chih.
8,9,Ciudad de México,C.D.Mx
9,10,Durango,Dgo.


# Todos los grupos

## Combinaciones

In [8]:
dist_all = endutih_vivhogar[["ENT","DOMINIO","Grupo", "FAC_HOG"]].groupby(["ENT","DOMINIO","Grupo"]).agg("sum").reset_index().rename(columns={"FAC_HOG":"Total"})
dist_all["ENT"] = dist_all.ENT.replace({codigo:nombre for codigo,nombre in zip(estados["Codigo"], estados["ENT"])},regex=True)
dist_all["DOMINIO"] = dist_all.DOMINIO.replace({"R":"Rural", "U":"Urbano"},regex=True)
dist_all["perc"] = (dist_all.Total/ dist_all.Total.sum()*100)
dist_all

,ENT,DOMINIO,Grupo,Total,perc
0,Ags.,Rural,20,86605,0.242699
1,Ags.,Rural,21,507,0.001421
2,Ags.,Rural,22,473,0.001326
3,Ags.,Rural,23,272,0.000762
4,Ags.,Urbano,20,113614,0.318389
...,...,...,...,...,...
515,Zac.,Urbano,32,149,0.000418
516,Zac.,Urbano,33,80273,0.224955
517,Zac.,Urbano,34,384,0.001076
518,Zac.,Urbano,42,31107,0.087173


## Distribución

In [9]:
dist_grupos = endutih_vivhogar[["ESTRATO","Grupo", "FAC_HOG"]].groupby(["ESTRATO","Grupo"]).agg("sum").reset_index()
dist_grupos

,ESTRATO,Grupo,FAC_HOG
0,1,11,6984839
1,2,20,17295201
2,2,21,108498
3,2,22,110877
4,2,23,92468
5,3,30,138234
6,3,31,81353
7,3,32,104798
8,3,33,7106718
9,3,34,43151


In [10]:
dist_grupos.Grupo.nunique()

15

Relativos al total de hogares

In [11]:
(dist_grupos/ dist_grupos.FAC_HOG.sum()*100)

,ESTRATO,Grupo,FAC_HOG
0,0.000003,0.000031,19.574120
1,0.000006,0.000056,48.467593
2,0.000006,0.000059,0.304052
3,0.000006,0.000062,0.310719
4,0.000006,0.000064,0.259130
5,0.000008,0.000084,0.387383
6,0.000008,0.000087,0.227981
7,0.000008,0.000090,0.293683
8,0.000008,0.000092,19.915670
9,0.000008,0.000095,0.120925


## Concentración en estados

In [12]:
grupo_estados = pd.pivot_table(data=endutih_vivhogar, columns="ENT", values= "FAC_HOG", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_estados.columns = estados.ENT
grupo_estados

ENT              Ags.    B.C.   B.C.S   Camp.   Coah.    Col.   Chis.   Chih.  \
ESTRATO Grupo                                                                   
1       11          0   52722   40490   94288   36088    6864  829585   85504   
2       20     200219  507326  111115  116090  400247  137829  464702  478467   
        21       1519    4220     846    1099    1979     572    2155       0   
        22       2593    2880    1160    1355    1994    1319    3528     626   
        23       1420    1709     684    1789    3652    1373    2577       0   
3       30       2227    7336    2127     513    6509     393     785    1650   
        31       2643    3634     674     515    1848     673    2034    1705   
        32        769    5485     105     245     537     212       0    1053   
        33     122136  334985   54272   21061  347796   35622   55827  398314   
        34        909    1279     680     399    1891     335       0     525   
4       40       1304    4844    1187     487    1238    1761       0     912   
        41       1413     670     394     156     550     582     677     549   
        42      45140  181461   34926   17174   98480   41080   20260  160656   
        43       1660    1246     102     145    2668     141     681     840   
        44        822    1462     139     116       0     283       0       0   

ENT             C.D.Mx    Dgo.    Gto.    Gro.    Hgo.     Jal.  Edo. Méx.  \
ESTRATO Grupo                                                                
1       11        4040   67991  244784  571453  221026    90597     589821   
2       20     1089994  284133  918089  323185  537452  1146740    2608220   
        21       11364     265    7161    1917    2819     7753      28016   
        22        3704     670   12024    2474    2361     5509      13199   
        23         908     238    9479    1125    2225     5185       2385   
3       30       32531       0    9714     454       0    16543      14126   
        31       12437     802    3978    1180     409     7568       9700   
        32       41001       0    4820       0     853    13531       4978   
        33     1193378  101901  274208   53578   41641   733640     719961   
        34        6117     203    5216       0       0     9146          0   
4       40       14164    1111    5134       0       0     6530      11313   
        41        6802     842    1038       0       0    11843          0   
        42      462092   47411  103019    1848   38859   274829     587679   
        43       28215       0    6096       0       0    15569      22043   
        44        1971       0    3003       0       0        0          0   

ENT             Mich.    Mor.    Nay.    N.L.    Oax.    Pue.    Qro.  \
ESTRATO Grupo                                                           
1       11     294591   68508   37842  137973  679375  575579  260256   
2       20     811481  371788  249027  625256  414561  816830  225819   
        21       6156    2935     834     926    1151     843    2803   
        22       5496    3714     767    3429    2029    5337    2703   
        23      18619    2069    1075       0    2006       0    1593   
3       30       2586    2686     809    9802    1842       0    1273   
        31       3511    1685     730    5068       0    1083    1458   
        32       1401    2179     302    5453     461    3306    2321   
        33     181981   73897   64788  516119   40322  225656   84143   
        34       2187     825     326     752     518       0     680   
4       40       1996       0     466   11848       0    1086    1299   
        41       1383     517       0    2597       0    2212     686   
        42      35488   22677   17266  321829    6538  105479   74424   
        43          0    2090     440   10032       0    2835    1408   
        44          0     108     179       0       0       0    1588   

ENT            Q. Roo.   S.L.P    Sin.    Son.    Tab

### Relativo al grupo

In [13]:
grupo_estados_rel_grupo = grupo_estados.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_estados_rel_grupo.style.format(precision=4)

De forma general, parece que hay una buena distribución de los grupos en los estados

### Relativo a la Entidad

In [14]:
grupo_estados_rel_ent = grupo_estados.apply(lambda x: x.div(x.sum()), axis=0)*100
grupo_estados_rel_ent.style.format(precision=4)

In [15]:
np.amax(grupo_estados_rel_ent)

C:\Users\jazmi\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


ENT
Ags.         52.035481
B.C.         45.653264
B.C.S        44.642247
Camp.        45.448495
Coah.        44.202890
Col.         60.177088
Chis.        59.992653
Chih.        42.312219
C.D.Mx       41.027628
Dgo.         56.200860
Gto.         57.103503
Gro.         59.699607
Hgo.         63.405317
Jal.         48.901847
Edo. Méx.    56.559761
Mich.        59.367565
Mor.         66.907094
Nay.         66.433596
N.L.         37.869424
Oax.         59.137642
Pue.         46.937617
Qro.         39.286652
Q. Roo.      57.382214
S.L.P        35.339037
Sin.         50.241358
Son.         43.645319
Tab.         57.578661
Tamps.       54.322547
Tlax.        81.637524
Ver.         50.260302
Yuc.         42.175304
Zac.         59.793218
dtype: float64

La mayoría se encuentra en el grupo 20, que es donde está casi la mitad de los hogares mexicanos

## Internet

In [16]:
grupo_internet = pd.pivot_table(data=endutih_vivhogar, columns="P4_4", values= "FAC_HOG", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_internet.columns = pd.Series(["Sí", "No"], name="Internet en el hogar")
grupo_internet

Internet en el hogar        Sí       No
ESTRATO Grupo                          
1       11             1767283  5217556
2       20            10168088  7127113
        21              108134      364
        22              110877        0
        23               47795    44673
3       30              125409    12825
        31               59912    21441
        32              104798        0
        33             5561698  1545020
        34               43151        0
4       40               75734     2710
        41               37829     9095
        42             3055407   314417
        43              106893        0
        44               15830        0

### Relativo al grupo

In [17]:
grupo_internet_rel_grupo = grupo_internet.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_internet_rel_grupo.style.format(precision=4)

Hay variaciones dentro de los grupos, sin embargo, destaca el hecho de que hay grupos donde todos los hogares tienen Internet

### Relativo a la Disponibilidad de internet

In [18]:
grupo_internet_rel_int = grupo_internet.apply(lambda x: x.div(x.sum()), axis=0)*100
grupo_internet_rel_int.style.format(precision=4)

### Internet Detalles

In [19]:
grupo_internet_tipo = pd.pivot_table(data=endutih_vivhogar.loc[endutih_vivhogar.P4_4==1], columns="P4_5", values= "FAC_HOG", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_internet_tipo.columns = pd.Series(["Fija", "Móvil", "Ambas", "No sabe"], name="Tipo de Internet en el hogar")
grupo_internet_tipo

Tipo de Internet en el hogar     Fija   Móvil    Ambas  No sabe
ESTRATO Grupo                                                  
1       11                    1129636  415595   214295     7757
2       20                    6785857  768901  2576932    36398
        21                      76096     750    27372     3916
        22                      84507       0    23244     3126
        23                      36662    3387     5906     1840
3       30                      78400    4079    42930        0
        31                      42975     452    14191     2294
        32                      56224       0    48574        0
        33                    3290073  187424  2076314     7887
        34                      26185       0    15539     1427
4       40                      40709     221    34804        0
        41                      17155     156    20518        0
        42                    1831189   54078  1160692     9448
        43                      48549    1740    56604        0
        44                       9351       0     6479        0

#### Relativo al grupo

In [20]:
grupo_internet_tipo_rel_grupo = grupo_internet_tipo.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_internet_tipo_rel_grupo.style.format(precision=4)

De los hogares que tienen internet, en su mayoría lo tienen de forma fija

#### Relativo al tipo de Internet

In [21]:
grupo_internet_tipo_rel_tipo = grupo_internet_tipo.apply(lambda x: x.div(x.sum()), axis=0)*100
grupo_internet_tipo_rel_tipo.style.format(precision=4)

## Electricidad

In [22]:
grupo_electricidad = pd.pivot_table(data=endutih_vivhogar, columns="P1_4", values= "FAC_HOG", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_electricidad.columns = pd.Series(["Sí", "No"], name="Electricidad")
grupo_electricidad

Electricidad         Sí      No
ESTRATO Grupo                  
1       11      6870009  114830
2       20     17249295   45906
        21       108498       0
        22       110877       0
        23        92468       0
3       30       138234       0
        31        81353       0
        32       104798       0
        33      7092239   14479
        34        43151       0
4       40        78444       0
        41        46924       0
        42      3368593    1231
        43       106893       0
        44        15830       0

### Relativo al grupo

In [23]:
grupo_electricidad_rel_grupo = grupo_electricidad.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_electricidad_rel_grupo.style.format(precision=4)

Como la mayoría de los hogares tienen electricidad, no consideraremos este factor como una limitante

## Computadora

In [24]:
endutih_vivhogar["COMPU"] = (endutih_vivhogar[["P4_2_1", "P4_2_2", "P4_2_3"]]==1).any(axis=1)

In [25]:
grupo_compu = pd.pivot_table(data=endutih_vivhogar, columns="COMPU", values= "FAC_HOG", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_compu.columns = pd.Series(["No", "Sí"], name="Computadora en el hogar")
grupo_compu

Computadora en el hogar        No       Sí
ESTRATO Grupo                             
1       11                5874989  1109850
2       20               10410499  6884702
        21                  28615    79883
        22                  48555    62322
        23                  54205    38263
3       30                  41005    97229
        31                  36945    44408
        32                  22979    81819
        33                2825919  4280799
        34                  16688    26463
4       40                  16268    62176
        41                  13491    33433
        42                 666910  2702914
        43                   7685    99208
        44                   4009    11821

### Relativo al grupo

In [26]:
grupo_compu_rel_grupo = grupo_compu.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_compu_rel_grupo.style.format(precision=4)

Vemos que en su mayoría no cuentan con un equipo de cómputo

### Detalles

In [27]:
grupo_compu_tipo = endutih_vivhogar[(endutih_vivhogar.COMPU==True)&(endutih_vivhogar.P4_2_1==1)][["ESTRATO", "Grupo", "FAC_HOG"]].groupby(["ESTRATO", "Grupo"]).agg("sum")
grupo_compu_tipo.columns = ["Ordenador"]
grupo_compu_tipo

Ordenador
ESTRATO Grupo           
1       11        198187
2       20       2082084
        21         36247
        22         17407
        23          8069
3       30         47775
        31         13896
        32         34766
        33       1480728
        34          8115
4       40         25260
        41          5319
        42       1029916
        43         42334
        44          5032

In [28]:
grupo_compu_tipo["Laptop"] = endutih_vivhogar[(endutih_vivhogar.COMPU==True)&(endutih_vivhogar.P4_2_2==1)][["ESTRATO", "Grupo", "FAC_HOG"]].groupby(["ESTRATO", "Grupo"]).agg("sum")
grupo_compu_tipo["Tablet"] = endutih_vivhogar[(endutih_vivhogar.COMPU==True)&(endutih_vivhogar.P4_2_3==1)][["ESTRATO", "Grupo", "FAC_HOG"]].groupby(["ESTRATO", "Grupo"]).agg("sum")
grupo_compu_tipo

Ordenador   Laptop   Tablet
ESTRATO Grupo                             
1       11        198187   785620   353407
2       20       2082084  4828180  2437367
        21         36247    45183    26237
        22         17407    36152    21112
        23          8069    24177    13594
3       30         47775    65313    39595
        31         13896    31700    19362
        32         34766    65720    32345
        33       1480728  3282566  1621163
        34          8115    19804    12515
4       40         25260    55985    27349
        41          5319    29013    10505
        42       1029916  2242544  1263455
        43         42334    86069    54325
        44          5032    11051     5890

In [29]:
grupo_compu_tipo_rel_grupo = grupo_compu_tipo/np.array(grupo_compu["Sí"]).repeat(3).reshape(15,3)
grupo_compu_tipo_rel_grupo

Ordenador    Laptop    Tablet
ESTRATO Grupo                               
1       11      0.178571  0.707861  0.318428
2       20      0.302422  0.701291  0.354027
        21      0.453751  0.565615  0.328443
        22      0.279307  0.580084  0.338757
        23      0.210883  0.631864  0.355278
3       30      0.491366  0.671744  0.407234
        31      0.312917  0.713835  0.436003
        32      0.424914  0.803236  0.395324
        33      0.345900  0.766812  0.378706
        34      0.306655  0.748366  0.472924
4       40      0.406266  0.900428  0.439864
        41      0.159094  0.867795  0.314211
        42      0.381039  0.829676  0.467442
        43      0.426720  0.867561  0.547587
        44      0.425683  0.934862  0.498266

Las laptops son claramente dominates a comparación de las otras opciones

## Número de integrantes Promedio*
*Mayores de 15 años

In [30]:
integrantes = endutih_res[["ESTRATO", "Grupo", "UPM", "VIV_SEL", "HOGAR", "FAC_HOGAR"]].groupby(["ESTRATO", "Grupo", "UPM", "VIV_SEL", "HOGAR"]).agg(["count"])
integrantes.columns = ["Total"]

In [31]:
endutih_vivhogar["Integrantes_total"] = np.array(integrantes["Total"])*endutih_vivhogar["FAC_HOG"]

In [32]:
grupo_integrantes = endutih_vivhogar[["ESTRATO", "Grupo", "Integrantes_total"]].groupby(["ESTRATO", "Grupo"]).agg("sum")
grupo_integrantes = grupo_integrantes.div(dist_grupos.FAC_HOG, axis=0)
grupo_integrantes

ValueError: cannot join with no overlapping index names

De forma general, parece que se tomaron 3 residentes por hogar

## Línea telefónica

In [ ]:
grupo_tel = pd.pivot_table(data=endutih_vivhogar, columns="P5_5", values= "FAC_HOG", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_tel.columns = pd.Series(["Sí", "No"], name="Línea telefónica en el hogar")
grupo_tel

### Relativo al grupo

In [ ]:
grupo_tel_rel_grupo = grupo_tel.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_tel_rel_grupo.style.format(precision=4)

Llama la atención cómo la línea telefónica no es tan usada como el Internet

## Actividad Laboral

In [ ]:
grupo_trabajo = pd.pivot_table(data=endutih_res, columns="P3_10", values= "FAC_HOGAR", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_trabajo.columns = pd.Series(["Trabajó","No fue al trabajo", "Buscó trabajo", "Está jubilado", "Se dedicó a estudiar", "Quehaceres del hogar", "Limitación para trabajar", "No trabajó"], name="Actividad laboral")
grupo_trabajo

### Relativo al grupo

In [ ]:
grupo_trabajo_rel_grupo = grupo_trabajo.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_trabajo_rel_grupo.style.format(precision=4)

De manera general, podemos ver que la mayoría d elos integrantes de los grupos son población económicamente activa, sin embargo, llama la atención que en el grupo 11, haya un alto porcentaje de residentes que se dedican a los quehaceres del hogar

## EDUCACIÓN

In [ ]:
grupo_edu = pd.pivot_table(data=endutih_res, columns=["NIVEL"], values= "FAC_HOGAR", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_edu.columns = pd.Series(["Ninguno", "Preescolar", "Primaria", "Secundaria","Normal básica", "Estudio técnico", "Preparatoria", "Estudio técnico superior", "Licenciatura o ingeniería", "Especialidad", "Maestría", "Doctorado", "No sabe"], name="Nivel Educativo")
grupo_edu

### Relativo al grupo

In [ ]:
grupo_edu_rel_grupo = grupo_edu.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_edu_rel_grupo.style.format(precision=4)

## Celular

In [ ]:
grupo_cel = pd.pivot_table(data=endutih_vivhogar, columns="P4_1_6", values= "FAC_HOG", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_cel.columns = pd.Series(["Sí", "No"], name="Celular en el hogar")
grupo_cel

### Relativo al grupo

In [ ]:
grupo_cel_rel_grupo = grupo_cel.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_cel_rel_grupo.style.format(precision=4)

### Detalles

In [ ]:
endutih_vivhogar["Movil"] = endutih_vivhogar["FAC_HOG"]*endutih_vivhogar["P4_1_6_1"]
endutih_vivhogar["Smartphone"] = endutih_vivhogar["FAC_HOG"]*endutih_vivhogar["P4_1_6_2"]

In [ ]:
grupo_cel_tipo = endutih_vivhogar[(endutih_vivhogar.P4_1_6==1)][["ESTRATO", "Grupo", "FAC_HOG",]].groupby(["ESTRATO", "Grupo"]).agg("sum")
grupo_cel_tipo.columns = ["Total"]
grupo_cel_tipo["Móvil"]= endutih_vivhogar[(endutih_vivhogar.P4_1_6==1)&(endutih_vivhogar.P4_1_6_1>0)][["ESTRATO", "Grupo", "Movil",]].groupby(["ESTRATO", "Grupo"]).agg("sum")
grupo_cel_tipo["Smartphone"] = endutih_vivhogar[(endutih_vivhogar.P4_1_6==1)&(endutih_vivhogar.P4_1_6_2>0)][["ESTRATO", "Grupo", "Smartphone"]].groupby(["ESTRATO", "Grupo"]).agg("sum")
grupo_cel_tipo

In [ ]:
grupo_cel_tipo_promedio = grupo_cel_tipo[["Móvil", "Smartphone"]].div(grupo_cel_tipo.Total, axis=0)
grupo_cel_tipo_promedio

## Sexo

In [ ]:
grupo_sexo = pd.pivot_table(data=endutih_res, index=["ESTRATO","Grupo"], columns="SEXO", values="FAC_HOGAR", aggfunc="sum")
grupo_sexo.columns = pd.Series(["Hombre","Mujer"], name="SEXO")
grupo_sexo

### Relativo al grupo

In [ ]:
grupo_sexo_rel_grupo = grupo_sexo.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_sexo_rel_grupo

## IDTMex

In [33]:
IDTMex.IDTMex_desglosado(endutih_completa)

{'acceso': 0.5852823902958105,
 'uso': 0.4840102760430942,
 'aptitudes': 0.5330448624089406,
 'IDTMex': 5.3432603901735}

### Grupos Estado

In [35]:
IDTMex_grupos_edo = pd.pivot_table(data=endutih_completa, columns="ENT", index=["ESTRATO","Grupo"], aggfunc=IDTMex.IDTMex,)
IDTMex_grupos_edo.columns = estados.ENT
IDTMex_grupos_edo

C:\Users\jazmi\AppData\Local\Temp\ipykernel_9548\4058064193.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  IDTMex_grupos_edo = pd.pivot_table(data=endutih_completa, columns="ENT", index=["ESTRATO","Grupo"], aggfunc=IDTMex.IDTMex,)


ENT                Ags.      B.C.     B.C.S     Camp.     Coah.       Col.  \
ESTRATO Grupo                                                                
1       11          NaN  3.509422  4.322510  3.152224  2.249533   3.963927   
2       20     4.715311  5.865413  5.453550  5.451482  4.782666   5.155523   
        21     7.212964  8.055016  8.173970  8.091542  6.232668   7.447854   
        22     6.318044  6.640156  6.356524  5.950369  5.743930   6.035900   
        23     4.545903  4.770804  5.154370  4.265302  4.278083   5.150232   
3       30     7.340089  7.944129  7.368104  7.188754  6.801749   7.772515   
        31     5.516981  7.971836  6.597414  6.250826  4.375062   6.547206   
        32     7.064853  8.008826  5.000000  7.897286  8.444444   8.324895   
        33     6.177313  7.108091  6.580328  6.732477  6.197529   6.695836   
        34     6.090581  8.034558  6.377292  7.475884  7.510594   5.652965   
4       40     6.939488  8.812334  8.475832  5.957107  8.075184   8.754785   
        41     7.994415  9.466667  6.858870  8.666667  9.666667   7.674468   
        42     7.388430  7.831090  7.710550  7.000035  7.836227   7.442446   
        43     8.464286  8.916199  7.666667  7.333333  8.008005  10.000000   
        44     6.579902  7.751601  7.666667  6.166667       NaN   8.566833   

ENT               Chis.     Chih.    C.D.Mx      Dgo.      Gto.      Gro.  \
ESTRATO Grupo                                                               
1       11     2.136097  2.704921  4.913072  3.019904  3.679311  3.401621   
2       20     4.516427  5.196660  6.676073  4.547788  4.341657  5.353832   
        21     6.791951       NaN  7.723988  7.666667  7.163437  6.664675   
        22     6.536865  7.833333  6.546627  5.737500  6.787817  6.436400   
        23     3.061872       NaN  4.666667  2.888889  3.965488  4.573409   
3       30     6.333333  6.371514  8.303545       NaN  7.598358  1.666667   
        31     6.963058  5.290554  7.253774  6.461579  6.536398  5.698692   
        32          NaN  8.198852  8.796732       NaN  8.150335       NaN   
        33     6.618886  6.295855  7.448001  6.276785  6.313263  7.533533   
        34          NaN  9.000000  8.591047  5.833333  6.578192       NaN   
4       40          NaN  9.000000  9.154944  7.809745  8.356411       NaN   
        41     7.666667  9.000000  6.904792  5.390363  7.666667       NaN   
        42     6.848023  7.667511  8.488164  7.075716  7.688000  8.416667   
        43     8.666667  4.000000  9.404851       NaN  9.086759       NaN   
        44          NaN       NaN  7.888889       NaN  6.818362       NaN   

ENT                Hgo.      Jal.  Edo. Méx.     Mich.      Mor.       Nay.  \
ESTRATO Grupo                                                                 
1       11     2.955906  2.926440   3.197817  2.357045  4.041712   2.402178   
2       20     4.667045  4.989033   5.654172  4.491235  5.307903   4.775072   
        21     6.520644  7.873503   7.621675  7.308558  7.332781   8.003444   
        22     4.786023  6.831785   6.162905  5.713394  5.696650   6.529955   
        23     5.007581  5.635662   4.666667  4.397901  3.773863   3.631431   
3       30          NaN  7.181075   7.299473  7.746778  6.993305   8.871010   
        31     1.666667  3.916692   4.486712  6.556072  6.016331   5.599994   
        32     7.127682  7.438901   7.932486  6.969107  8.165602   8.222222   
        33     6.425261  6.164887   6.861031  5.999945  6.379561   6.985303   
        34          NaN  5.983468        NaN  5.749542  6.522020   6.406769   
4       40          NaN  9.590595   7.092048  7.974406       NaN   8.138770   
        41          NaN  7.935612        NaN  5.417016  7.345476        NaN   
        42     7.234888  7.905820   7.828485  7.390869  7.408084   8.047967   
        43          NaN  8.970157   8.671252       NaN  8.507569   8.666667   
        44          NaN       NaN        NaN       NaN  5.666667  10.000000   

ENT                

In [36]:
IDTMex_grupos_edo.to_csv(base_path+"IDTMex_grupos_edo.csv")

## Desglose

In [37]:
IDTMex_grupos_edo_des = pd.pivot_table(data=endutih_completa, columns="ENT", index=["ESTRATO","Grupo"], aggfunc=IDTMex.IDTMex_desglosado,)
IDTMex_grupos_edo_des.columns = estados.ENT
IDTMex_grupos_edo_des

C:\Users\jazmi\AppData\Local\Temp\ipykernel_9548\3528744705.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  IDTMex_grupos_edo_des = pd.pivot_table(data=endutih_completa, columns="ENT", index=["ESTRATO","Grupo"], aggfunc=IDTMex.IDTMex_desglosado,)


ENT                                                         Ags.  \
ESTRATO Grupo                                                      
1       11                                                   NaN   
2       20     {'acceso': 0.5307600677258402, 'uso': 0.416396...   
        21     {'acceso': 0.9731731402238315, 'uso': 0.562970...   
        22     {'acceso': 0.6346895487851909, 'uso': 0.655043...   
        23     {'acceso': 0.5149647887323944, 'uso': 0.393306...   
3       30     {'acceso': 0.9066008082622363, 'uso': 0.568978...   
        31     {'acceso': 0.6919220582671206, 'uso': 0.434221...   
        32     {'acceso': 0.8852405721716515, 'uso': 0.587341...   
        33     {'acceso': 0.7090026691556953, 'uso': 0.527489...   
        34     {'acceso': 0.6003850385038504, 'uso': 0.665849...   
4       40     {'acceso': 0.8021472392638036, 'uso': 0.552481...   
        41     {'acceso': 0.9115357395612173, 'uso': 0.652450...   
        42     {'acceso': 0.8364366415595924, 'uso': 0.629220...   
        43     {'acceso': 1.0, 'uso': 0.6666666666666666, 'ap...   
        44     {'acceso': 0.6870437956204379, 'uso': 0.666666...   

ENT                                                         B.C.  \
ESTRATO Grupo                                                      
1       11     {'acceso': 0.3828288001213914, 'uso': 0.282572...   
2       20     {'acceso': 0.6270386299933377, 'uso': 0.596713...   
        21     {'acceso': 0.9327606635071091, 'uso': 0.779643...   
        22     {'acceso': 0.6019097222222223, 'uso': 0.812000...   
        23     {'acceso': 0.5825043885313049, 'uso': 0.467043...   
3       30     {'acceso': 0.9118729552889858, 'uso': 0.723827...   
        31     {'acceso': 0.8262933406714364, 'uso': 0.757099...   
        32     {'acceso': 0.9157247037374658, 'uso': 0.803412...   
        33     {'acceso': 0.7652447124498112, 'uso': 0.712498...   
        34     {'acceso': 0.75, 'uso': 0.8259056554599948, 'a...   
4       40     {'acceso': 0.9644921552436003, 'uso': 0.788545...   
        41     {'acceso': 1.0, 'uso': 1.0, 'aptitudes': 0.733...   
        42     {'acceso': 0.8246441384099062, 'uso': 0.774750...   
        43     {'acceso': 1.0, 'uso': 0.8167469234884965, 'ap...   
        44     {'acceso': 0.75, 'uso': 0.8358413132694938, 'a...   

ENT                                                        B.C.S  \
ESTRATO Grupo                                                      
1       11     {'acceso': 0.4331439861694245, 'uso': 0.432049...   
2       20     {'acceso': 0.5729356972505962, 'uso': 0.528366...   
        21     {'acceso': 0.95774231678487, 'uso': 0.71670606...   
        22     {'acceso': 0.6586206896551724, 'uso': 0.672325...   
        23     {'acceso': 0.5113304093567251, 'uso': 0.530723...   
3       30     {'acceso': 0.8533145275035261, 'uso': 0.641861...   
        31     {'acceso': 0.7117952522255193, 'uso': 0.687386...   
        32     {'acceso': 0.75, 'uso': 0.3333333333333333, 'a...   
        33     {'acceso': 0.7067594708136793, 'uso': 0.624262...   
        34     {'acceso': 0.6147058823529412, 'uso': 0.668382...   
4       40     {'acceso': 0.9427127211457456, 'uso': 0.776748...   
        41     {'acceso': 0.6611675126903553, 'uso': 0.661608...   
        42     {'acceso': 0.8238203630533127, 'uso': 0.722540...   
        43     {'acceso': 0.75, 'uso': 0.6666666666666666, 'a...   
        44     {'acceso': 0.75, 'uso': 0.6666666666666666, 'a...   

ENT                                                        Camp.  \
ESTRATO Grupo                                                      
1       11     {'acceso': 0.34052583573731543, 'uso': 0.23846...   
2       20     {'acceso': 0.6075114135584461, 'uso': 0.477881...   
        21     {'acceso': 0.9699727024567789, 'uso': 0.640467...   
        22     {'acceso': 0.6313653136531365, 'uso': 0.577475...   
        23     {'acceso': 0.4207657909446618, 'uso': 0.366175...   
3       30     {'acceso': 0.7558479532163742, 'uso': 

In [38]:
IDTMex_grupos_edo_des_long = IDTMex_grupos_edo_des.reset_index().melt(id_vars=["ESTRATO","Grupo"], value_vars=estados.ENT, var_name="ENT",value_name="IDT_dicci",)
IDTMex_grupos_edo_des_long

,ESTRATO,Grupo,ENT,IDT_dicci
0,1,11,Ags.,NaN
1,2,20,Ags.,"{'acceso': 0.5307600677258402, 'uso': 0.416396..."
2,2,21,Ags.,"{'acceso': 0.9731731402238315, 'uso': 0.562970..."
3,2,22,Ags.,"{'acceso': 0.6346895487851909, 'uso': 0.655043..."
4,2,23,Ags.,"{'acceso': 0.5149647887323944, 'uso': 0.393306..."
...,...,...,...,...
475,4,40,Zac.,NaN
476,4,41,Zac.,NaN
477,4,42,Zac.,"{'acceso': 0.8050358440222457, 'uso': 0.756772..."
478,4,43,Zac.,"{'acceso': 1.0, 'uso': 1.0, 'aptitudes': 0.5, ..."


In [50]:
conteos = endutih_completa[["ESTRATO","Grupo","ENT", "FAC_HOGAR"]].groupby(["ESTRATO","ENT","Grupo"]).agg("sum").reset_index()
conteos["ENT"] = conteos["ENT"].apply(lambda x: estados.loc[estados.Codigo==x,"ENT"].unique()[0])
conteos

,ESTRATO,ENT,Grupo,FAC_HOGAR
0,1,B.C.,11,202619
1,1,B.C.S,11,142486
2,1,Camp.,11,370557
3,1,Coah.,11,133295
4,1,Col.,11,22732
...,...,...,...,...
420,4,Yuc.,42,211094
421,4,Yuc.,43,4635
422,4,Yuc.,44,4848
423,4,Zac.,42,94796


In [51]:
IDTMex_des_long_fixed = IDTMex_grupos_edo_des_long.join(pd.json_normalize(IDTMex_grupos_edo_des_long['IDT_dicci'])).drop('IDT_dicci', axis='columns')
#IDTMex_des_long_fixed["IDTMex"] =  (0.4*IDTMex_des_long_fixed.acceso + 0.4*IDTMex_des_long_fixed.uso + 0.2*IDTMex_des_long_fixed.aptitudes)*10
IDTMex_des_long_fixed = IDTMex_des_long_fixed.dropna(subset=["acceso","uso","aptitudes","IDTMex"])
IDTMex_des_long_fixed = pd.merge(IDTMex_des_long_fixed, conteos, on=["ESTRATO","Grupo", "ENT"])
IDTMex_des_long_fixed


,ESTRATO,Grupo,ENT,acceso,uso,aptitudes,IDTMex,FAC_HOGAR
0,2,20,Ags.,0.530760,0.416397,0.463342,4.715311,800380
1,2,21,Ags.,0.973173,0.562971,0.534194,7.212964,6503
2,2,22,Ags.,0.634690,0.655043,0.579556,6.318044,8468
3,2,23,Ags.,0.514965,0.393307,0.456408,4.545903,5262
4,3,30,Ags.,0.906601,0.568979,0.718886,7.340089,7891
...,...,...,...,...,...,...,...,...
420,3,32,Zac.,0.750000,0.500000,0.500000,6.000000,298
421,3,33,Zac.,0.719707,0.623086,0.640362,6.651896,274852
422,3,34,Zac.,0.847005,0.586430,0.666667,7.067073,619
423,4,42,Zac.,0.805036,0.756772,0.789014,7.825261,94796


In [52]:
IDTMex_des_long_fixed.to_csv(base_path+"long_IDTMex_desglosado_grupos_edo.csv")

In [59]:
IDTMex_des_long_fixed[(IDTMex_des_long_fixed['ENT']=="C.D.Mx")&(IDTMex_des_long_fixed["Grupo"]==23)]

,ESTRATO,Grupo,ENT,acceso,uso,aptitudes,IDTMex,FAC_HOGAR
115,2,23,C.D.Mx,0.5,0.333333,0.666667,4.666667,1816


In [68]:
IDTMex_des_long_fixed[(IDTMex_des_long_fixed['ENT']=="Son.")]

,ESTRATO,Grupo,ENT,acceso,uso,aptitudes,IDTMex,FAC_HOGAR
329,1,11,Son.,0.419049,0.421256,0.423469,4.208157,284257
330,2,20,Son.,0.572233,0.662649,0.477237,5.894004,1338278
331,2,21,Son.,0.864572,0.941438,0.558562,8.341163,2772
332,2,22,Son.,0.750000,0.912889,0.554771,7.761097,8386
333,2,23,Son.,1.000000,0.833333,0.500000,8.333333,824
334,3,30,Son.,0.856419,0.859030,0.684561,8.230919,11047
335,3,31,Son.,0.417434,0.696670,0.665061,5.786540,1868
336,3,32,Son.,0.857607,0.955034,0.778919,8.808401,5960
337,3,33,Son.,0.693434,0.806902,0.601713,7.204768,919578
338,3,34,Son.,0.750000,0.960036,0.507861,7.855865,3720


### Dominio Grupos Estado

In [ ]:
IDTMex_dominio_grupos_edo = pd.pivot_table(data=endutih_completa, columns="ENT", index=["ESTRATO","DOMINIO","Grupo"], aggfunc=IDTMex.IDTMex,)
IDTMex_dominio_grupos_edo.columns = estados.ENT
IDTMex_dominio_grupos_edo

In [ ]:
IDTMex_dominio_grupos_edo.to_csv(base_path+"IDTMex_dominio_grupos_edo.csv")

#### Desglose

In [ ]:
IDTMex_dominio_grupos_edo_des = pd.pivot_table(data=endutih_completa, columns="ENT", index=["ESTRATO","DOMINIO","Grupo"], aggfunc=IDTMex.IDTMex_desglosado,)
IDTMex_dominio_grupos_edo_des.columns = estados.ENT
IDTMex_dominio_grupos_edo_des

In [ ]:
IDTMex_dominio_grupos_edo_des_long = IDTMex_dominio_grupos_edo_des.reset_index().melt(id_vars=["ESTRATO","DOMINIO","Grupo"], value_vars=estados.ENT, var_name="ENT",value_name="IDT_dicci",)
IDTMex_dominio_grupos_edo_des_long

In [ ]:
conteos = endutih_completa[["ESTRATO","DOMINIO","Grupo", "FAC_HOGAR"]].groupby(["ESTRATO","DOMINIO","Grupo"]).agg("sum").reset_index()
conteos

In [ ]:
IDTMex_des_long_fixed = IDTMex_dominio_grupos_edo_des_long.join(pd.json_normalize(IDTMex_dominio_grupos_edo_des_long['IDT_dicci'])).drop('IDT_dicci', axis='columns')
#IDTMex_des_long_fixed["IDTMex"] =  (0.4*IDTMex_des_long_fixed.acceso + 0.4*IDTMex_des_long_fixed.uso + 0.2*IDTMex_des_long_fixed.aptitudes)*10
IDTMex_des_long_fixed = IDTMex_des_long_fixed.dropna(subset=["acceso","uso","aptitudes","IDTMex"])
IDTMex_des_long_fixed = pd.merge(IDTMex_des_long_fixed, conteos, on=["ESTRATO","DOMINIO","Grupo"])
IDTMex_des_long_fixed


In [ ]:
IDTMex_des_long_fixed.to_csv(base_path+"long_IDTMex_desglosado_dominio_grupos_edo.csv")